In [2]:
from dotenv import load_dotenv
import os
from pymongo import MongoClient
from collections import defaultdict
from math import sqrt

In [3]:
# 데이터베이스 연결
load_dotenv()
client = MongoClient(os.getenv('DB_ADR'),
          username=os.getenv('DB_USER'),
          password=os.getenv('DB_PASSWORD'),
          authSource=os.getenv('DB_AuthSource'),
          authMechanism=os.getenv('DB_AuthMechanism'))
db = client.get_database(os.getenv('DB_Collection'))

'''
가장 많은 로그가 있는 사용자 top 5
uuid                            
bfe7f9b54d5641c8a1f77d8c51e46fcf    9620
d3d9a8ead8d8423592cc6fbce311d80d    9457
5965657bbb344c62bd49af4256d05ec1    9110
99a8eb4b813747bbb870043a96695735    8890
020450eed61f4daaa07a47ae3bcda750    8633
'''

# 특정 유저의 frequent_zone이 0인 basic-unit 찾기
user_uuid = "18be9796f0804f6b9e9044c7f89dd590"


In [4]:
# 특정 유저의 주요 지역 확인
collection_filtered = db['basicunit_clusterlabel_frequency_up100_20-24_0402']

frequent_zone_1 = collection_filtered.find({"uuid": user_uuid, "frequent_zone": 1}, {"destination_area": 1, "cluster_label":1, "_id": 0})
frequent_zone_0 = collection_filtered.find({"uuid": user_uuid, "frequent_zone": 0}, {"destination_area": 1, "cluster_label":1, "_id": 0})

frequent_zone_1=list(frequent_zone_1)
frequent_zone_0=list(frequent_zone_0)


In [5]:
# 특정 유저의 frequent_zone이 0인 basic-unit 찾기
collection_filtered = db['basicunit_clusterlabel_frequency_up100_20-24_0402']

frequent_zone_1 = collection_filtered.find({"uuid": user_uuid, "frequent_zone": 1}, {"destination_area": 1, "cluster_label":1, "_id": 0})
frequent_zone_0 = collection_filtered.find({"uuid": user_uuid, "frequent_zone": 0}, {"destination_area": 1, "cluster_label":1, "_id": 0})

mapped_category_collection = db['user_category_frequency_20-24']
user_where_collection = db['user_category_place_20-24']

# 특정 유저의 mapped_category 및 선호도 // place name 및 mapped category 조회
target_user_categories = list(mapped_category_collection.find({"uuid": user_uuid}, {"mapped_category": 1, "frequency": 1, "_id": 0}))
target_user_where = list(user_where_collection.find({"uuid": user_uuid}, {"mapped_category": 1, "place_name": 1, "_id": 0}))

frequent_zone_1=list(frequent_zone_1)
frequent_zone_0=list(frequent_zone_0)


In [6]:
frequent_zone_1

[{'destination_area': '서울 관악구', 'cluster_label': 2},
 {'destination_area': '서울 금천구', 'cluster_label': 1},
 {'destination_area': '서울 구로구', 'cluster_label': 3},
 {'destination_area': '서울 영등포구', 'cluster_label': 2},
 {'destination_area': '서울 금천구', 'cluster_label': 2},
 {'destination_area': '서울 관악구', 'cluster_label': 1},
 {'destination_area': '서울 관악구', 'cluster_label': 4},
 {'destination_area': '서울 동작구', 'cluster_label': 2}]

In [7]:
# 특정 유저가 방문한 지역과 로컬들
cluster_uuids = {}

for zone in frequent_zone_1:
    busic_unit = zone["destination_area"]
    cluster = zone["cluster_label"]
    
    query = {
        "destination_area": busic_unit,
        "cluster_label": cluster,
        "frequent_zone": 1
    }
    projection = {
        "_id": 0,
        "uuid": 1,
    }
    docs = list(collection_filtered.find(query, projection))

    filtered_docs = [doc for doc in docs if doc.get("uuid") != user_uuid]
    if filtered_docs:
        key = (busic_unit, cluster)
        if key not in cluster_uuids:
            cluster_uuids[key] = []
        cluster_uuids[key].extend(docs)

cluster_uuids

{('서울 관악구', 2): [{'uuid': '01958421c1d1474095c369faa6c9a30e'},
  {'uuid': '02439b75b4b5446892cf0562680e83c2'},
  {'uuid': '06def1c9e43a42fba8d5ad608ea6d688'},
  {'uuid': '07c5c0460b064a5ba452f41b1bcc22bc'},
  {'uuid': '09556ccdf4b6428987cd89dffcca6bdd'},
  {'uuid': '0aa455a0b1a04049bab94182b1d9d5a5'},
  {'uuid': '0c3b8790e9e34aa7bf2a26f0aec882d0'},
  {'uuid': '0c4d37de103e4218be798c31a6c544cf'},
  {'uuid': '0f0ba0a032a745b4a3f75b5036b589ca'},
  {'uuid': '1206ab61ee8444e2b56dafd9372abc8a'},
  {'uuid': '13bf22f2293545f4b0d43a9e57774ce1'},
  {'uuid': '166419300d47493da5c37f206b273f74'},
  {'uuid': '169cde03c41340e8a96664b0b7b1509d'},
  {'uuid': '18be9796f0804f6b9e9044c7f89dd590'},
  {'uuid': '1b12ae6947d641988ecf5331461c580d'},
  {'uuid': '1d0d5db5dd76400daad6e80550b0e66d'},
  {'uuid': '1da36d0d23314729bdb09d267cf5a659'},
  {'uuid': '1e20dad1df1a4608aa6b475225ae8b6e'},
  {'uuid': '1e998ea63c774f41943be6a2e4ed6877'},
  {'uuid': '1ebf80d165974c50b5c8252f20d768d3'},
  {'uuid': '20b9601d063a4

In [8]:
cluster_uuids.keys()

dict_keys([('서울 관악구', 2), ('서울 금천구', 1), ('서울 구로구', 3), ('서울 영등포구', 2), ('서울 금천구', 2), ('서울 관악구', 1), ('서울 관악구', 4), ('서울 동작구', 2)])

In [9]:
len(cluster_uuids[('서울 금천구', 1)])

204

In [10]:
# 테스트를 위해 한 구역 지정
target_key = ('서울 금천구', 1)  # 남기고 싶은 키

cluster_uuids = {
    key: value
    for key, value in cluster_uuids.items()
    if key == target_key
}

In [11]:
cluster_uuids

{('서울 금천구', 1): [{'uuid': '0037b972742344bb8bba86dd835bda73'},
  {'uuid': '02ae2b0222d041a1b0dc467fddff9762'},
  {'uuid': '03ae55f496df408ebf10fffe2da5aa0b'},
  {'uuid': '04bb2a7f184c4b1e8bd4c20c4209eee4'},
  {'uuid': '052b3e03e6a74007a5450f110f7d849a'},
  {'uuid': '05311fc081444960858ee88cfdc5b4d0'},
  {'uuid': '078e281d08d646d9a73516fdf1826b6b'},
  {'uuid': '08c0a343aafd402e81b537f002452753'},
  {'uuid': '08cd2b3ed8644847a21fb67babfe43c7'},
  {'uuid': '0afd94a1a766404f89bdacc10f42207f'},
  {'uuid': '0c2ff73b62f54e3a9f148a23b35e6f76'},
  {'uuid': '0dd6bf5b0743447b9d3ad6001869d650'},
  {'uuid': '0f0ba0a032a745b4a3f75b5036b589ca'},
  {'uuid': '0f76c242cf784a38ad628b85dfd18b6c'},
  {'uuid': '1088ba5c826248d48177c2fd98e1df26'},
  {'uuid': '138542d8d29f423b9f24bda60ceb7af2'},
  {'uuid': '13f01814fd174cf4af52e440f41559d6'},
  {'uuid': '15abce93012f4b069523123053a19c2f'},
  {'uuid': '166419300d47493da5c37f206b273f74'},
  {'uuid': '17ba0fe0cc2d4f1cae328fa5baead1c1'},
  {'uuid': '189bea0297a24

In [12]:
user_category_place_20_24 = db['user_category_place_20-24']
rating_details_2020_2024 = db['rating_details_2020-2024']

# 해당 지역, 클러스터에 있는 지역민들이 방문한 식당 정리
cluster_restuarant_list = {}

for (basic_unit, cluster), uuids in cluster_uuids.items():
    result_docs = []

    for uuid_dict in uuids:
        for _, uuid in uuid_dict.items():

    # Step 1: user_category_place_20_24에서 uuid 찾기
            docs1 = user_category_place_20_24.find({"uuid": uuid},{"_id": 0, "uuid": 1, "place_name": 1, "mapped_category": 1})
            
            for doc1 in docs1:
                place = doc1["place_name"]

    # Step 2: rating_details_2020-2024에서 place_name 일치 + road_address_name에 지역 포함
                query2 = {
                    "place_name": place,
                    "road_address_name": {"$regex": basic_unit}
                }
                projection2 = {
                    "_id": 0,
                    "place_name": 1,
                    "road_address_name": 1,
                    "x": 1,
                    "y": 1,
                    "rate": 1
                }

                matching_docs2 = rating_details_2020_2024.find(query2, projection2)

    # Step 3: 문서 조합하여 결과 문서 생성
                for doc2 in matching_docs2:
                    combined_doc = {
                        "uuid": doc1["uuid"],
                        "place_name": doc1["place_name"],
                        "road_address_name": doc2["road_address_name"],
                        "mapped_category": doc1["mapped_category"],
                        "x": doc2["x"],
                        "y": doc2["y"],
                        "rate": doc2["rate"]
                    }
                    result_docs.append(combined_doc)

    # Step 4: 결과 저장
    if result_docs:
        cluster_restuarant_list[(basic_unit, cluster)] = result_docs

In [13]:
cluster_restuarant_list #사용자들이 방문한 장소 추출

{('서울 금천구',
  1): [{'uuid': '05311fc081444960858ee88cfdc5b4d0',
   'place_name': '학커피&호프',
   'road_address_name': '서울 금천구 독산로 54',
   'mapped_category': '술집',
   'x': '126.90621377156',
   'y': '37.4519594774089',
   'rate': '2.8'}, {'uuid': '078e281d08d646d9a73516fdf1826b6b',
   'place_name': '펀비어킹 금천시흥은행나무사거리점',
   'road_address_name': '서울 금천구 금하로24길 8',
   'mapped_category': '술집',
   'x': '126.908444168926',
   'y': '37.4505664577521',
   'rate': '4.9'}, {'uuid': '078e281d08d646d9a73516fdf1826b6b',
   'place_name': '펀비어킹 금천시흥은행나무사거리점',
   'road_address_name': '서울 금천구 금하로24길 8',
   'mapped_category': '술집',
   'x': '126.908444168926',
   'y': '37.4505664577521',
   'rate': '4.9'}, {'uuid': '078e281d08d646d9a73516fdf1826b6b',
   'place_name': '펀비어킹 금천시흥은행나무사거리점',
   'road_address_name': '서울 금천구 금하로24길 8',
   'mapped_category': '술집',
   'x': '126.908444168926',
   'y': '37.4505664577521',
   'rate': '4.9'}, {'uuid': '078e281d08d646d9a73516fdf1826b6b',
   'place_name': '제주오전복',
   'road

In [14]:
#클러스터 중심점과 식당의 좌표를 비교할 함수
def euclidean_distance(x1, y1, x2, y2):
    return sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

# 방문 지역의 장소에 클러스터 매핑
cluster_info_collection = db["cluster_info"]

# cluster_restuarant_list에서 'cluster_label': '레이블' 필드 추가
for (basic_unit, cluster), docs in cluster_restuarant_list.items():
    
    # Step 1: cluster_info에서 해당 지역(basic_unit) 문서 가져오기
    cluster_doc = cluster_info_collection.find_one({"basic-unit": basic_unit})
    if not cluster_doc:
        continue
    center_points = cluster_doc["center_points"]

    for doc in docs:
        x, y = float(doc["x"]), float(doc["y"])
        
        closest_cluster_num = None
        min_distance = float('inf')

        # Step 2: 각 center point와 유클리드 거리 계산
        for cluster_num, (lat, lng) in center_points.items():
            distance = euclidean_distance(x, y, float(lng), float(lat))
            if distance < min_distance:
                min_distance = distance
                closest_cluster_num = int(cluster_num)

        # Step 3: 가장 가까운 클러스터 번호를 'cluster_label': '번호' 형태로 추가
        if closest_cluster_num is not None:
            doc["cluster_label"] = closest_cluster_num
cluster_restuarant_list

{('서울 금천구',
  1): [{'uuid': '05311fc081444960858ee88cfdc5b4d0',
   'place_name': '학커피&호프',
   'road_address_name': '서울 금천구 독산로 54',
   'mapped_category': '술집',
   'x': '126.90621377156',
   'y': '37.4519594774089',
   'rate': '2.8',
   'cluster_label': 1}, {'uuid': '078e281d08d646d9a73516fdf1826b6b',
   'place_name': '펀비어킹 금천시흥은행나무사거리점',
   'road_address_name': '서울 금천구 금하로24길 8',
   'mapped_category': '술집',
   'x': '126.908444168926',
   'y': '37.4505664577521',
   'rate': '4.9',
   'cluster_label': 1}, {'uuid': '078e281d08d646d9a73516fdf1826b6b',
   'place_name': '펀비어킹 금천시흥은행나무사거리점',
   'road_address_name': '서울 금천구 금하로24길 8',
   'mapped_category': '술집',
   'x': '126.908444168926',
   'y': '37.4505664577521',
   'rate': '4.9',
   'cluster_label': 1}, {'uuid': '078e281d08d646d9a73516fdf1826b6b',
   'place_name': '펀비어킹 금천시흥은행나무사거리점',
   'road_address_name': '서울 금천구 금하로24길 8',
   'mapped_category': '술집',
   'x': '126.908444168926',
   'y': '37.4505664577521',
   'rate': '4.9',
   'cluster

In [15]:
len(cluster_restuarant_list[('서울 금천구',1)])

306

In [16]:
# 해당 문서에서 사용자와 지역민을 구분!

local_people_dict = {}      #지역민 방문장소
user_dict={}            #사용자 방문장소

for key, doc_list in cluster_restuarant_list.items():
    # user_uuid와 다른 문서들만 필터링
    filtered_docs=[]
    user_docs=[]
    for doc in doc_list:
        if doc.get("uuid") != user_uuid:
            filtered_docs.append(doc)
        else:
            user_docs.append(doc)
    
    # 필터링 결과가 있다면 새로운 딕셔너리에 추가
    if filtered_docs:
        local_people_dict[key] = filtered_docs
    if user_docs:
        user_dict[key] = user_docs

In [17]:
len(user_dict[('서울 금천구',1)])

29

In [18]:
len(local_people_dict[('서울 금천구',1)])

277

In [19]:
# 지역민 중복되는 장소 제거
deduped_result_dict = {}

for key, doc_list in local_people_dict.items():
    seen_place_names = set()
    unique_docs = []

    for doc in doc_list:
        place = doc.get("place_name")
        if place and place not in seen_place_names:
            seen_place_names.add(place)
            unique_docs.append(doc)

    deduped_result_dict[key] = unique_docs
deduped_result_dict

{('서울 금천구',
  1): [{'uuid': '05311fc081444960858ee88cfdc5b4d0',
   'place_name': '학커피&호프',
   'road_address_name': '서울 금천구 독산로 54',
   'mapped_category': '술집',
   'x': '126.90621377156',
   'y': '37.4519594774089',
   'rate': '2.8',
   'cluster_label': 1}, {'uuid': '078e281d08d646d9a73516fdf1826b6b',
   'place_name': '펀비어킹 금천시흥은행나무사거리점',
   'road_address_name': '서울 금천구 금하로24길 8',
   'mapped_category': '술집',
   'x': '126.908444168926',
   'y': '37.4505664577521',
   'rate': '4.9',
   'cluster_label': 1}, {'uuid': '078e281d08d646d9a73516fdf1826b6b',
   'place_name': '제주오전복',
   'road_address_name': '서울 금천구 디지털로10길 9',
   'mapped_category': '분식',
   'x': '126.889114803475',
   'y': '37.4776198829081',
   'rate': '4.0',
   'cluster_label': 2}, {'uuid': '078e281d08d646d9a73516fdf1826b6b',
   'place_name': '달달보드레 가산점',
   'road_address_name': '서울 금천구 디지털로10길 9',
   'mapped_category': '기타 > 푸드코트',
   'x': '126.889056014524',
   'y': '37.4776198277227',
   'rate': '4.7',
   'cluster_label': 2}

In [ ]:
# basic-unit 식당 중 클러스터 안에 있는 지역만 추출하고, rating 순으로 정렬
restuarant_list = {}

for (basic_unit, cluster), doc_list in deduped_result_dict.items():
    matched_docs = []

    for doc in doc_list:
        cluster_label = doc.get("cluster_label", "")
        if cluster_label == cluster:
            matched_docs.append(doc)

    # rating 기준 내림차순 정렬
    matched_docs.sort(key=lambda x: float(x.get("rate", 0)), reverse=True)

    restuarant_list[(basic_unit, cluster)]=(matched_docs)

# 지역민 기준 상위 10개 식당 출력
top_10_restuarant_list=defaultdict(list)
for key, doc_list in restuarant_list.items():
    for doc in doc_list:
        top_10_restuarant_list[key].append({'place_name':doc['place_name'], 'rate':doc['rate'],'mapped_category':doc['mapped_category']})
        print(f"지역: {key[0]}, 클러스터: {key[1]} → 평점: {doc['rate']}, 식당이름: {doc['place_name']}, 카테고리: {doc['mapped_category']}")
# print(restuarant_list[(basic_unit, cluster)][])



지역: 서울 금천구, 클러스터: 1 → 평점: 5.0, 식당이름: 명랑핫도그 금천시흥점, 카테고리: 패스트푸드
지역: 서울 금천구, 클러스터: 1 → 평점: 5.0, 식당이름: 수수미감자탕, 카테고리: 한식 > 탕류
지역: 서울 금천구, 클러스터: 1 → 평점: 5.0, 식당이름: 도복상회 도도한떡볶이 금천시흥점, 카테고리: 한식
지역: 서울 금천구, 클러스터: 1 → 평점: 5.0, 식당이름: 부가네얼큰이 금천시흥점, 카테고리: 한식 > 구이 & 육류 요리
지역: 서울 금천구, 클러스터: 1 → 평점: 5.0, 식당이름: 대성숯불바베큐왕족발, 카테고리: 한식 > 구이 & 육류 요리
지역: 서울 금천구, 클러스터: 1 → 평점: 5.0, 식당이름: 퓨전와사비, 카테고리: 일식
지역: 서울 금천구, 클러스터: 1 → 평점: 5.0, 식당이름: 곰비임비호프, 카테고리: 술집
지역: 서울 금천구, 클러스터: 1 → 평점: 5.0, 식당이름: 청년피자 금천구점, 카테고리: 피자
지역: 서울 금천구, 클러스터: 1 → 평점: 5.0, 식당이름: KNC 숯불바베큐치킨호프 시흥점, 카테고리: 치킨
지역: 서울 금천구, 클러스터: 1 → 평점: 4.9, 식당이름: 펀비어킹 금천시흥은행나무사거리점, 카테고리: 술집
지역: 서울 금천구, 클러스터: 1 → 평점: 4.9, 식당이름: 한방삼계명가 시흥점, 카테고리: 한식 > 구이 & 육류 요리
지역: 서울 금천구, 클러스터: 1 → 평점: 4.8, 식당이름: 파삼쭈삼 시흥점, 카테고리: 한식
지역: 서울 금천구, 클러스터: 1 → 평점: 4.7, 식당이름: 본전찾은전집, 카테고리: 술집
지역: 서울 금천구, 클러스터: 1 → 평점: 4.7, 식당이름: 지도리 시흥본점, 카테고리: 치킨
지역: 서울 금천구, 클러스터: 1 → 평점: 4.7, 식당이름: 61번가금화설렁탕, 카테고리: 한식 > 탕류
지역: 서울 금천구, 클러스터: 1 → 평점: 4.6, 식당이름: 마약떡볶이랑불오뎅 시흥본점, 카테고리: 분식
지역: 서울 금천구, 클러스

In [38]:
from collections import defaultdict

# 지역민 기준 상위 10개 식당 출력
top_10_restuarant_list = defaultdict(list)

for key, doc_list in restuarant_list.items():
    # 상위 10개만 추출
    top_docs = doc_list[:10]

    for doc in top_docs:
        top_10_restuarant_list[key].append({
            'place_name': doc['place_name'],
            'rate': doc['rate'],
            'mapped_category': doc['mapped_category']
        })

        print(f"지역: {key[0]}, 클러스터: {key[1]} → 평점: {doc['rate']}, 식당이름: {doc['place_name']}, 카테고리: {doc['mapped_category']}")


지역: 서울 금천구, 클러스터: 1 → 평점: 5.0, 식당이름: 명랑핫도그 금천시흥점, 카테고리: 패스트푸드
지역: 서울 금천구, 클러스터: 1 → 평점: 5.0, 식당이름: 수수미감자탕, 카테고리: 한식 > 탕류
지역: 서울 금천구, 클러스터: 1 → 평점: 5.0, 식당이름: 도복상회 도도한떡볶이 금천시흥점, 카테고리: 한식
지역: 서울 금천구, 클러스터: 1 → 평점: 5.0, 식당이름: 부가네얼큰이 금천시흥점, 카테고리: 한식 > 구이 & 육류 요리
지역: 서울 금천구, 클러스터: 1 → 평점: 5.0, 식당이름: 대성숯불바베큐왕족발, 카테고리: 한식 > 구이 & 육류 요리
지역: 서울 금천구, 클러스터: 1 → 평점: 5.0, 식당이름: 퓨전와사비, 카테고리: 일식
지역: 서울 금천구, 클러스터: 1 → 평점: 5.0, 식당이름: 곰비임비호프, 카테고리: 술집
지역: 서울 금천구, 클러스터: 1 → 평점: 5.0, 식당이름: 청년피자 금천구점, 카테고리: 피자
지역: 서울 금천구, 클러스터: 1 → 평점: 5.0, 식당이름: KNC 숯불바베큐치킨호프 시흥점, 카테고리: 치킨
지역: 서울 금천구, 클러스터: 1 → 평점: 4.9, 식당이름: 펀비어킹 금천시흥은행나무사거리점, 카테고리: 술집


In [24]:
# print(top_10_restuarant_list)

In [36]:
# 방문자 중복되는 장소 제거
u_deduped_result_dict = {}

for key, doc_list in user_dict.items():
    seen_place_names = set()
    unique_docs = []

    for doc in doc_list:
        place = doc.get("place_name")
        if place and place not in seen_place_names:
            seen_place_names.add(place)
            unique_docs.append(doc)

    u_deduped_result_dict[key] = unique_docs
# u_deduped_result_dict

u_deduped_result_dict_filter=defaultdict(list)
for key, doc_list in u_deduped_result_dict.items():
    for doc in doc_list:
        u_deduped_result_dict_filter[key].append({'place_name':doc['place_name'],'mapped_category':doc['mapped_category']})
        print(f"지역: {key[0]}, 클러스터: {key[1]} → 평점: {doc['rate']}, 식당이름: {doc['place_name']}, 카테고리: {doc['mapped_category']}")

지역: 서울 금천구, 클러스터: 1 → 평점: 4.0, 식당이름: 빙동댕, 카테고리: 간식 > 디저트
지역: 서울 금천구, 클러스터: 1 → 평점: 4.0, 식당이름: 원푸드코리아 가산점, 카테고리: 한식
지역: 서울 금천구, 클러스터: 1 → 평점: 3.0, 식당이름: 처갓집양념치킨 시흥점, 카테고리: 치킨
지역: 서울 금천구, 클러스터: 1 → 평점: 4.6, 식당이름: 개성만점족발 웅이푸드, 카테고리: 한식 > 구이 & 육류 요리
지역: 서울 금천구, 클러스터: 1 → 평점: 5.0, 식당이름: 야구장떡볶이, 카테고리: 분식 > 떡볶이
지역: 서울 금천구, 클러스터: 1 → 평점: 5.0, 식당이름: 황금가마솥곰탕, 카테고리: 한식 > 탕류
지역: 서울 금천구, 클러스터: 1 → 평점: 5.0, 식당이름: 고향통닭, 카테고리: 치킨
지역: 서울 금천구, 클러스터: 1 → 평점: 4.6, 식당이름: 신미경정통춘천닭갈비 독산직영점, 카테고리: 한식 > 구이 & 육류 요리
지역: 서울 금천구, 클러스터: 1 → 평점: 4.7, 식당이름: 본전찾은전집, 카테고리: 술집
지역: 서울 금천구, 클러스터: 1 → 평점: 4.6, 식당이름: 용뽀끼, 카테고리: 분식 > 떡볶이
지역: 서울 금천구, 클러스터: 1 → 평점: 5.0, 식당이름: 볶찜 가산점, 카테고리: 한식 > 구이 & 육류 요리
지역: 서울 금천구, 클러스터: 1 → 평점: 4.3, 식당이름: 먹자골등갈비찌개마을, 카테고리: 한식 > 구이 & 육류 요리
지역: 서울 금천구, 클러스터: 1 → 평점: 5.0, 식당이름: 메고지고 구로독산동점, 카테고리: 간식
지역: 서울 금천구, 클러스터: 1 → 평점: 2.8, 식당이름: 알통떡강정 서울 금천센터점, 카테고리: 간식 > 치킨 & 스낵류
지역: 서울 금천구, 클러스터: 1 → 평점: 4.4, 식당이름: 이화룡불짬뽕, 카테고리: 중식 > 중국요리
지역: 서울 금천구, 클러스터: 1 → 평점: 3.0, 식당이름: 엄마의마음분식, 카테고리: 분식
지역: 서울 금

In [27]:
len(u_deduped_result_dict_filter[('서울 금천구', 1)])

18

In [41]:
# 식당 기준으로 일치하는 곳

matched_docs = []

# 1. u_deduped_result_dict의 모든 place_name을 집합으로 수집
place_names_from_u_deduped_result_dict_filter = set()
for doc_list in u_deduped_result_dict_filter.values():
    for doc in doc_list:
        place_name = doc.get("place_name")
        if place_name:
            place_names_from_u_deduped_result_dict_filter.add(place_name)

# 2. top_10_restuarant_list에서 place_name이 일치하는 문서를 찾음
for doc_list in top_10_restuarant_list.values():
    for doc in doc_list:
        if doc.get("place_name") in place_names_from_u_deduped_result_dict_filter:
            matched_docs.append(doc)

# 결과 출력
for i, doc in enumerate(matched_docs, 1):
    print(f"{i}. {doc}")

1. {'place_name': '대성숯불바베큐왕족발', 'rate': '5.0', 'mapped_category': '한식 > 구이 & 육류 요리'}


In [39]:
# 추천된 장소들의 카테고리
recommended_categories = []
for doc_list in top_10_restuarant_list.values():
    for doc in doc_list:
        category = doc.get("mapped_category")
        if category:
            recommended_categories.append(category)

# 실제 방문한 장소들의 카테고리
visited_categories = []
for doc_list in u_deduped_result_dict_filter.values():
    for doc in doc_list:
        category = doc.get("mapped_category")
        if category:
            visited_categories.append(category)

# 결과 확인
print("추천된 카테고리 수:", len(recommended_categories))
print("방문한 카테고리 수:", len(visited_categories))

추천된 카테고리 수: 10
방문한 카테고리 수: 18


In [40]:
from collections import Counter

# 각 카테고리 등장 횟수 세기
rec_counter = Counter(recommended_categories)
visit_counter = Counter(visited_categories)

# 일치하는 카테고리 수 계산 (교집합의 합)
common_count = sum(min(rec_counter[cat], visit_counter[cat]) for cat in rec_counter if cat in visit_counter)

# 총 개수
total_rec = sum(rec_counter.values())
total_visit = sum(visit_counter.values())

# Precision, Recall, F1 계산
precision = common_count / total_rec if total_rec > 0 else 0
recall = common_count / total_visit if total_visit > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

# 출력
print(f"✔ Precision: {precision:.4f}")
print(f"✔ Recall:    {recall:.4f}")
print(f"✔ F1 Score:  {f1_score:.4f}")


✔ Precision: 0.6000
✔ Recall:    0.3333
✔ F1 Score:  0.4286
